In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
df=pd.read_csv('/kaggle/input/superstore-sales/Superstore.csv',encoding='latin1')
df.head(4)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2013-138688,13-06-2013,17-06-2013,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2012-108966,11-10-2012,18-10-2012,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310


In [57]:
print(f'data shape',df.shape)
df.describe()

data shape (9994, 21)


,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [58]:
df.duplicated().sum()

0

In [59]:
df=df.drop(['Row ID','Order ID','Customer ID','Customer Name','Postal Code','Product ID'],axis=1)
df.columns=df.columns.str.lower()

In [60]:
df['order date']=pd.to_datetime(df['order date'],errors='coerce',dayfirst=True)
df['ship date']=pd.to_datetime(df['ship date'],errors='coerce',dayfirst=True)

df['order year']=df['order date'].dt.year
df['order month']=df['order date'].dt.month
df['order day']=df['order date'].dt.day
df['order dayofweek']=df['order date'].dt.dayofweek
df['day of shipping']=df['ship date'].dt.day
df['year of shipping']=df['ship date'].dt.year
df['month of shipping']=df['ship date'].dt.month
df['dayofweek of shipping']=df['ship date'].dt.dayofweek
df=df.drop(['order date','ship date'],axis=1)
df.head(2)

,ship mode,segment,country,city,state,region,category,sub-category,product name,sales,...,discount,profit,order year,order month,order day,order dayofweek,day of shipping,year of shipping,month of shipping,dayofweek of shipping
0,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,...,0.0,41.9136,2013,11,9,5,12,2013,11,1
1,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,...,0.0,219.5820,2013,11,9,5,12,2013,11,1


In [61]:
df['avg_discount_per_sale'] = df['discount'] / df['sales'].replace(0, np.nan)
df['sales']=np.log1p(df['sales'])
df['region_avg_profit'] = df.groupby('region')['profit'].transform('mean')

In [62]:
from sklearn.preprocessing import OrdinalEncoder

cat_cols = df.select_dtypes(include='object').columns.tolist()

encoder = OrdinalEncoder()
df[cat_cols] = encoder.fit_transform(df[cat_cols])


In [63]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb

X=df.drop(columns=['sales'])
y=df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', xgb.XGBRegressor(random_state=42))
])
pipeline.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [64]:
y_pred=pipeline.predict(X_test)

In [65]:
from sklearn.metrics import mean_squared_error,r2_score
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print(f'mse',mean_squared_error(y_test,y_pred))
print(f'r2_score',r2_score(y_test,y_pred))

mse 0.05412224921560014
r2_score 0.9787518752924779


In [66]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score( pipeline, X, y,scoring='neg_mean_squared_error',cv=5)

mse_scores = -scores
rmse_scores = np.sqrt(mse_scores)

print("Cross-Validated MSE Scores:", mse_scores)
print("Mean MSE:", mse_scores.mean())
print("Mean RMSE:", rmse_scores.mean())


Cross-Validated MSE Scores: [0.05758222 0.06862881 0.06689012 0.04542323 0.0545902 ]
Mean MSE: 0.05862291608265206
Mean RMSE: 0.24146758562559872


In [68]:
comparison_df = pd.DataFrame({
    'Actual Sales': y_test.values,
    'Predicted Sales': y_pred
}).reset_index(drop=True)

print(comparison_df.head(10))

   Actual Sales  Predicted Sales
0      6.336486         6.120512
1      3.628917         3.565184
2      3.645450         3.910401
3      5.361564         5.489681
4      5.149167         5.300552
5      4.606529         4.645870
6      5.832416         5.749583
7      2.911263         2.979341
8      3.162094         3.087222
9      4.285406         4.270354
